In [1]:
import numpy as np
import pandas as pd

In [2]:
def oneHot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [3]:
def getData():
    '''
    MNIST data set 
    x has 784 feature
    y is op value from 0 to 9 
    '''
    data = np.array(pd.read_csv('data/MNIST/MNIST_train.csv'))
    x = (data[:,1:]/255).T
    y = oneHot(data[:,0])
    return x,y
    

In [4]:
class Activation:
    @staticmethod
    def ReLU(Z, isDerivation=False):
        if isDerivation:
            return Z > 0
        return np.maximum(Z, 0)

    @staticmethod
    def sigmoid(Z, isDerivation=False):
        if isDerivation:
            op = Activation.sigmoid(Z)
            return op * (1- op)
        return 1/(1 + np.exp(-Z))

    @staticmethod
    def softmax(Z,isDerivation=False):
        if isDerivation:
            return 1
        Z = Z - np.max(Z, axis=0)
        A = np.exp(Z) / sum(np.exp(Z))
        return A

In [5]:
class Accuracy:
    @staticmethod
    def multiClass(target, prediction):
        return np.argmax(target, axis=0) == np.argmax(prediction, axis=0)

In [6]:
class Error:
    @staticmethod
    def meanSquareError(target, predicted, isDerivation=False):
        if isDerivation:
            return 2 * (predicted - target) / np.size(target)
        loss = np.power(target - predicted, 2)
        accuracy = Accuracy.multiClass(target, predicted)
        return {'loss': loss, 'accuracy': accuracy}
    
    @staticmethod
    def crossEntropyLoss(target, predicted,  isDerivation=False):
        if isDerivation:
            return predicted - target
        loss = -target * np.log(predicted + 10 ** -100)
        accuracy = Accuracy.multiClass(target,predicted)
        return {'loss': loss, 'accuracy': accuracy}
    
    @staticmethod
    def hiddenError(target, predicted):
        return target.T.dot(predicted)

In [7]:
class DenseLayer:
    op = []
    bias = []
    weight = []
    output = []
    isInput=False
    noOfNodes = 0
    activation = None
    def __init__(self, inputSize=0, outputSize=0, activation=None,isInput=False,input=[]):
        '''
        inputSize -> no.of.input feature 
        outputSize -> no.of.output
        '''
        if isInput:
            self.output = input
            self.isInput = True
            self.noOfNodes = input.shape[0]
        else :
            self.noOfNodes = inputSize
            self.activation = activation
            self.weight = np.random.rand(inputSize, outputSize) - 0.5
            self.bias = np.random.rand(inputSize,1) - 0.5

    def generateWeight(self,*r):
        '''
        Receive input as set that define the set shape
        '''
        return np.random.randn(*r) - 0.5

In [8]:
class NeuralNetwork:
    input = []
    target = []
    layers = []
    history = {'loss': [], 'accuracy':[]}
    loss = None

    def __init__(self, input, target, loss):
        self.loss = loss
        self.input = input
        self.target = target
        self.layers.append(DenseLayer(None,None,None,isInput=True, input=self.input))

    def append(self, node, activationFunction):
        preNode = self.layers[len(self.layers) -1]
        self.layers.append(DenseLayer(inputSize=node,outputSize=preNode.noOfNodes, activation=activationFunction))

    def train(self, epoch=100, lr=0.01):
        for i in range(epoch):
            for j in range(len(self.layers)):
                if not self.layers[j].isInput:
                    self.layers[j].op = self.forWord(self.layers[j], self.layers[j-1].output)
                    self.layers[j].output = self.layers[j].activation(self.layers[j].op)
                if j == len(self.layers)-1 :
                    loss = self.loss(self.target,self.layers[j].output)
                    self.history['loss'].append(np.mean(loss['loss']))
                    self.history['accuracy'].append(np.mean(loss['accuracy']))
                    # index = len(self.history['loss']) -1
                    print(f"Epoch: {i+1} Loss: {self.history['loss'][-1]} Accuracy: {self.history['accuracy'][-1]}")
                    # calculate loss
                    self.backPropogation(loss['loss'],lr)

    def forWord(self, layer, input):
        return np.dot(layer.weight, input) + layer.bias

    def backPropogation(self, loss,lr):
        layer_length = len(self.layers)
        for index, layer in enumerate(self.layers[::-1]):
            if not layer.isInput:
                pervious_node = self.layers[layer_length - index - 2]
                oldw = np.copy(layer.weight)
                layer.weight, layer.bias = self.gradient(
                    layer.weight, layer.bias, pervious_node.output, loss,lr)
                loss = Error.hiddenError(oldw, loss)

    def gradient(self, w, b, x, err, lr=0.01):
        w = w - (1/len(x[0]) * (err.dot(x.T))) * lr
        b = b - (lr * np.mean(err, axis=1).reshape(b.shape))
        return w,b        

In [9]:
x,y = getData()

In [10]:
model = None
model = NeuralNetwork(np.copy(x),np.copy(y),Error.crossEntropyLoss)

model.append(10, Activation.ReLU)
model.append(10,Activation.softmax)
model.train(10)

Epoch: 1 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 2 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 3 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 4 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 5 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 6 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 7 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 8 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 9 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762
Epoch: 10 Loss: 0.35405090024666674 Accuracy: 0.1031904761904762


In [11]:
model1 = NeuralNetwork(np.copy(x),np.copy(y),Error.meanSquareError)

model1.append(10, Activation.ReLU)
model1.append(10,Activation.softmax)
model1.train(10, 0.1)

Epoch: 1 Loss: 0.10460289562838226 Accuracy: 0.0670952380952381


ValueError: operands could not be broadcast together with shapes (10,10) (784,10) 

# Debug

In [27]:
def debug(epoch, loss, lr=0.1):
    hw = np.random.rand(10, 784) - 0.5
    hb = np.random.rand(10, 1) - 0.5
    ow = np.random.rand(10, 10) - 0.5
    ob = np.random.rand(10, 1) - 0.5

    def forWord(x,w,b):
        return np.dot(w, x) + b

    def backWord(err, w,b, x,lr = 0.01):
        m = 1/len(x[0])
        w = w - (m * (err.dot(x.T))) * lr
        b = b - (lr * np.mean(err, axis=1).reshape(b.shape))
        return w, b
    
    for i in range(epoch):
        
        #forword
        #Hidden
        hid_op = forWord(x,hw,hb)
        hid_act = Activation.ReLU(hid_op)

        #op layer
        op = forWord(hid_act,ow,ob)
        y_pred = Activation.softmax(op)

        # backword
        # output error
        op_err = loss(y, y_pred, True)
        ow,ob = backWord(op_err, ow, ob, hid_act, lr)

        # #hidden error
        hid_err = Error.hiddenError(ow,op_err) * Activation.ReLU(hid_op,True)
        hw,hb = backWord(hid_err, hw, hb, x, lr)

        err = loss(y, y_pred)
        print(f"Epoch : {i + 1}, Loss : {np.mean(err['loss'])}, Accuracy : {np.mean(err['accuracy'])}")

In [28]:
debug(100,Error.crossEntropyLoss,0.1)

Epoch : 1, Loss : 0.48327715779022967, Accuracy : 0.09114285714285714
Epoch : 2, Loss : 0.36077419788809156, Accuracy : 0.1044047619047619
Epoch : 3, Loss : 0.31645499197587157, Accuracy : 0.11719047619047619
Epoch : 4, Loss : 0.29324319185472597, Accuracy : 0.124
Epoch : 5, Loss : 0.27839410623951666, Accuracy : 0.12897619047619047
Epoch : 6, Loss : 0.26768106270724157, Accuracy : 0.13438095238095238
Epoch : 7, Loss : 0.25938369400096106, Accuracy : 0.1397857142857143
Epoch : 8, Loss : 0.2526721268507558, Accuracy : 0.14445238095238094
Epoch : 9, Loss : 0.24707817313720984, Accuracy : 0.1505
Epoch : 10, Loss : 0.2423105299922805, Accuracy : 0.15566666666666668
Epoch : 11, Loss : 0.2381784028828643, Accuracy : 0.16097619047619047
Epoch : 12, Loss : 0.23453608061688297, Accuracy : 0.1666190476190476
Epoch : 13, Loss : 0.23128051514721154, Accuracy : 0.1724047619047619
Epoch : 14, Loss : 0.2283263049899599, Accuracy : 0.17716666666666667
Epoch : 15, Loss : 0.22561363898383063, Accuracy :

In [29]:
debug(100,Error.meanSquareError,0.1)

Epoch : 1, Loss : 0.11869653794026085, Accuracy : 0.0884047619047619
Epoch : 2, Loss : 0.11869644640788884, Accuracy : 0.0884047619047619
Epoch : 3, Loss : 0.11869635487570838, Accuracy : 0.0884047619047619
Epoch : 4, Loss : 0.11869626334406605, Accuracy : 0.0884047619047619
Epoch : 5, Loss : 0.11869617181296181, Accuracy : 0.0884047619047619
Epoch : 6, Loss : 0.1186960802823957, Accuracy : 0.0884047619047619
Epoch : 7, Loss : 0.11869598875236766, Accuracy : 0.0884047619047619
Epoch : 8, Loss : 0.11869589722287772, Accuracy : 0.0884047619047619
Epoch : 9, Loss : 0.11869580569392589, Accuracy : 0.0884047619047619
Epoch : 10, Loss : 0.11869571416551218, Accuracy : 0.0884047619047619
Epoch : 11, Loss : 0.11869562263773854, Accuracy : 0.0884047619047619
Epoch : 12, Loss : 0.11869553111050966, Accuracy : 0.0884047619047619
Epoch : 13, Loss : 0.1186954395838189, Accuracy : 0.0884047619047619
Epoch : 14, Loss : 0.11869534805766617, Accuracy : 0.0884047619047619
Epoch : 15, Loss : 0.1186952565